In [ ]:
# Load the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.calibration import calibration_curve
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, f1_score, log_loss
from sklearn import decomposition
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import auc
import math
import joblib

In [ ]:
data = pd.read_csv("../input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv")

In [ ]:
data.head()

In [ ]:
data['Churn'].value_counts().plot.bar()
# We can see that the data set is imbalanced, i.e., not many people have churned.

In [ ]:
# Get the percentage of values in 'Churn' column
(data['Churn'].value_counts()/data['Churn'].count())*100

## We can see that the dataset is a little imbalanced. 
1. Churned customers = 26.53%
2. Existing customers = 73.46%

## We shall use SMOTE when fitting models for classification, and see if it provides any improvement.

# Data Pre-processing

In [ ]:
# Check if data has missing values
data.isnull().sum()

There are no null values in the dataset.

In [ ]:
data.dtypes

### In the dataset, the 'TotalCharges' column has numerical values, but its type is 'object'. So we will convert it to float.

In [ ]:
data.TotalCharges = pd.to_numeric(data.TotalCharges, errors='coerce')
data.isnull().sum()

### We see that there are 11 missing values in the TotalCharges column. When fitting models, we will:-
1. First split this data into test and train sets
2. Then impute the missing values in TotalCharges in the train set(if present in training set)
3. Fill the missing values in test set(if any)

# Exploratory Data Analysis

In [ ]:
# Separate out the churned and existing customers
churned = data.loc[data['Churn']=='Yes']
existing = data.loc[data['Churn']=='No']

In [ ]:
g = sns.catplot(x="gender", col="Churn",
                data=data, kind="count",
                height=5, aspect=0.8,palette='husl' );

### The genders are uniformally distributed in both churning and existing coustomers. Hence, gender cannot be considered a factor which determines whether a customer will churn

In [ ]:
g = sns.catplot(x="SeniorCitizen", col="Churn",
                data=data, kind="count",
                height=5, aspect=0.8, palette='YlOrBr');

In [ ]:
g = sns.catplot(x="Partner", col="Churn",
                data=data, kind="count",
                height=5, aspect=0.8);

### Most of the customers who have churned, did not have a partner.

## Analyze the numerical columns

In [ ]:
# Tenure vs Churn
plt.figure(figsize=(9, 4))
sns.boxplot(x="Churn", y="tenure",
            hue="Churn", palette="pastel",
            data=data).set_title("tenure vs Churn", fontsize=15)
sns.despine(offset=30, trim=True)

### We can see that 75% of the customers who churned(in the right boxplot), have stayed with the company for less than 30 months. 
### This is less than the median of existing customers group, which is around 37 months.
1. This tells us that most customers who left, did so relatively early.
2. There are some customers who left after long time, around 70 months(5.8 years). There are even some outliers, exceeding even that mark.

In [ ]:
# MonthlyCharges vs Churn
plt.figure(figsize=(9, 4))
sns.boxplot(x="Churn", y="MonthlyCharges",
            hue="Churn", palette='Blues',
            data=data, notch=True).set_title("MonthlyCharges vs Churn", fontsize=15)
sns.despine(offset=30, trim=True)

In [ ]:
# TotalCharges vs Churn
plt.figure(figsize=(9, 6))
sns.boxplot(x="Churn", y="TotalCharges",
            hue="Churn", palette="muted",
            data=data, notch=True).set_title("TotalCharges vs Churn", fontsize=15)
sns.despine(offset=30, trim=True)

### Many of the customers who left, were having high monthly charges. 109 customers are even outliers in Total Charges. 
There is probably some service which costed more, but did not provide the quality as expected by the customers. 


In [ ]:
churned_PS = churned["PhoneService"].value_counts()
existing_PS = existing["PhoneService"].value_counts()
colors = ['#99ff99','#ffcc99']
explode = (0, 0.1)
fig1, axs = plt.subplots(1, 2)

axs[0].pie(churned_PS, labels=churned_PS.index, autopct='%1.1f%%', shadow=True, startangle=100, colors=colors, explode=explode)
axs[0].set_title('Churned customers')

colors = ['#ff9999','#66b3ff']
axs[1].pie(existing_PS, labels=existing_PS.index, autopct='%1.1f%%', shadow=True, startangle=100, colors=colors, explode=explode)
axs[1].set_title('Existing customers')

plt.show()

### Both the existing and churned customers have neraly equal distribution for phone service.

In [ ]:
churned_IS = churned["InternetService"].value_counts()
existing_IS = existing["InternetService"].value_counts()

fig, axs = plt.subplots(1, 2)
fig.tight_layout()

axs[0].pie(churned_IS, labels=churned_IS.index, autopct='%1.2f%%', shadow=None)
axs[0].set_title('Churned customers')

axs[1].pie(existing_IS, labels=existing_IS.index, autopct='%1.2f%%', shadow=None)
axs[1].set_title('Existing customers')

plt.show()

In [ ]:
churned_IS

1. Almost 70% of the churned customers were using fiber optics for their internet service. Could this have to do something with the reason for their leaving the company services?
2. Also, nearly 94% customers who left, had even internet services. Could it be that they were dissatisfied by some of the services being provided over the internet?  

In [ ]:
fig, ax =plt.subplots(nrows=1,ncols=3,figsize = (14,4))
fig.tight_layout(pad=3.0)
sns.countplot(x ='MultipleLines', hue = "Churn", data = data, ax=ax[0],palette="rocket")
sns.countplot(x ='OnlineBackup', hue = "Churn", data = data, ax=ax[1],palette="rocket")
sns.countplot(x ='OnlineSecurity', hue = "Churn", data = data, ax=ax[2],palette="rocket")

fig, ax =plt.subplots(nrows=1,ncols=4,figsize = (14,4))
fig.tight_layout(pad=3.0)
sns.countplot(x ='DeviceProtection', hue = "Churn", data = data, ax=ax[0],palette="Set2")
sns.countplot(x ='TechSupport', hue = "Churn", data = data, ax=ax[1],palette="Set2")
sns.countplot(x ='StreamingTV', hue = "Churn", data = data, ax=ax[2],palette="Set2")
sns.countplot(x ='StreamingMovies', hue = "Churn", data = data, ax=ax[3],palette="Set2")

1. Many customers who left, did not have online backup, security, even though 94% had inernet services.
2. Device Protection and tech support was also not being used in the same proportion as the customers who have stayed.

In [ ]:
!pip install squarify

In [ ]:
import squarify
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
plt.subplots_adjust(top=0.95, bottom=0.05, left=0.05, right=0.95, hspace=0.35)

x1 = churned.groupby(['Contract']).MonthlyCharges.count().sort_values(ascending=False)
axes[0].set_title('Churned customers contract lengths')
squarify.plot(sizes=x1.tolist()[:3], label=x1.index.tolist()[:3], alpha=0.6, ax=axes[0])
axes[0].axis('off')
x2 = existing.groupby(['Contract']).MonthlyCharges.count().sort_values(ascending=False)
axes[1].set_title('Existing customers contract lengths')
squarify.plot(sizes=x2.tolist()[:3], label=x2.index.tolist()[:3], alpha=0.6, ax=axes[1])
axes[1].axis('off')
plt.show()

### We can see that a very large portion of the customers who left were mostly using month-to-month contracts. This probably made it easier for them to leave, as they were not bound to the company with long term contracts.

In [ ]:
#fig, axes = plt.subplots(1, 2, figsize=(14, 5))

churned_paymethods=churned["PaymentMethod"].value_counts()
existing_paymethods=existing["PaymentMethod"].value_counts()
    
names=churned_paymethods.index.tolist()
plt.pie(churned_paymethods, labels=names)
#plt.show()

# add a circle at the center
my_circle=plt.Circle((0,0), 0.7, color='white')
p=plt.gcf()
p.gca().add_artist(my_circle)
plt.title("Churned customer payment methods")
plt.show()

# Model Fitting for classification

In [ ]:
data.isnull().sum()

### We have 11 null values in 'TotalCharges' column. To impute them, we must first:-
1. Split into test and training.
2. Impute any missing values in training set.
3. Then fit the test data accordingly.

In [ ]:
# Split into test and training
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

data = data.drop(['customerID'], axis=1)
x = data.drop(['Churn'], axis=1)
y = data['Churn']
X_train, X_test, y_train, y_test = train_test_split(x, y, stratify=y, test_size=0.25, random_state=42)

# Replace the null values in 'TotalCharges' with mean value
train_mean = X_train['TotalCharges'].mean()
X_train['TotalCharges'].fillna(train_mean, inplace=True)

test_mean = X_test['TotalCharges'].mean()
X_test['TotalCharges'].fillna(test_mean, inplace=True)

In [ ]:
corr = X_train[['TotalCharges', 'MonthlyCharges', 'tenure']].corr()
# plot the heatmap
sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns)

1. Tenure has a very positive correlation with TotalCharges, but also has almost no correlation with MonthlyCharges.

## Borderline-SMOTE oversampling

Create the transformation pipeline

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
        ('std_scaler', StandardScaler()),
    ])

num_attribs = ['tenure', 'MonthlyCharges', 'TotalCharges']
cat_attribs = ['gender','SeniorCitizen','Partner','Dependents','PhoneService','MultipleLines','InternetService',
               'OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies',
               'Contract','PaperlessBilling','PaymentMethod']

# handle_unknown = 'ignore' is needed to produce transformed test data with same dimensions as the transformed training data.
full_pipeline = ColumnTransformer([
        ('num', num_pipeline, num_attribs),
        ("cat", OneHotEncoder(handle_unknown = 'ignore'), cat_attribs),
    ])

Transform the training data and test data

In [ ]:
data_prepared = full_pipeline.fit_transform(X_train)
test_data = full_pipeline.fit_transform(X_test) 

Perform the SMOTE oversampling

In [ ]:
# Install imblearn
import sys
!{sys.executable} -m pip install imblearn

from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import BorderlineSMOTE
from matplotlib import pyplot
from numpy import where

# summarize class distribution
counter = Counter(y_train)
print(counter)
# transform the dataset
oversample = BorderlineSMOTE(random_state=123)
X, y = oversample.fit_resample(data_prepared, y_train)
# summarize the new class distribution
counter = Counter(y)
print(counter)

### Our training data is well balanced now, containing equal samples from both classes(churned and existing customers)

## Principal Component Analysis to plot data

### Plot the original training data

In [ ]:
pca = decomposition.PCA(n_components=3)
pca.fit(data_prepared)
data_pca = pca.transform(data_prepared)
fig, ax = plt.subplots()
colors = {'Yes':'red','No':'blue'}
ax.scatter(data_pca[:,0], data_pca[:,1], c=y_train.map(colors))
plt.show()

1. The customer information are plotted w.r.t. to the first two principal components. We can see that there are very few customers who churned(red points).
2. We also see 2 distinct clusters for customers of both kind. This suggests that there must be some very specific behaviors belonging to customers in the clusters. 

### Plot the oversampled data

In [ ]:
pca = decomposition.PCA(n_components=3)
pca.fit(X)
X_pca = pca.transform(X)
fig, ax = plt.subplots()
colors = {'Yes':'red','No':'blue'}
y_series = pd.Series(y)
ax.scatter(X_pca[:,0], X_pca[:,1], c=y_series.map(colors))
plt.show()

1. We can see that the churned customers(red points) are in much larger number now.
2. Also, notice that the red points are generated in those places where they were in much larger density in the first plot. Hence, Borderline SMOTE makes sure not to generate outlier like samples, as it will increase noise in the dataset.

### Loading the models that I had already trained, so that they do not have to run again on Kaggle.

In [ ]:
# Load the models
RF_model = joblib.load("../input/pre-trained-models-for-customer-churn/telco_RF_model.pkl")
knn_model = joblib.load("../input/pre-trained-models-for-customer-churn/telco_knn_model.pkl")
log_model = joblib.load("../input/pre-trained-models-for-customer-churn/telco_logistic_model.pkl")
nb_model = joblib.load("../input/pre-trained-models-for-customer-churn/telco_naive_bayes_model.pkl")

RF_basemodel = joblib.load("../input/pre-trained-models-for-customer-churn/telco_RF_basemodel.pkl")
knn_basemodel = joblib.load("../input/pre-trained-models-for-customer-churn/telco_knn_basemodel.pkl")
log_basemodel = joblib.load("../input/pre-trained-models-for-customer-churn/telco_logistic_basemodel.pkl")
nb_basemodel = joblib.load("../input/pre-trained-models-for-customer-churn/telco_naive_bayes_basemodel.pkl")

# ## Since it is more important to classify the customers who might churn, we will focus more on precision and recall of the models, and select the final model based on that.

## Support Vector Machine classifier

In [ ]:
def tune_svm(x,y):
  param_grid = {'C': [0.1,1, 10], 'gamma': [1,0.1,0.01,0.001],'kernel': ['linear', 'rbf']}
  grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=2)
  grid.fit(x,y)
  return grid

svm_model = tune_svm(X,y)

In [ ]:
joblib.dump(svm_model, "telco_svm_model.pkl")

In [ ]:
y_pred = svm_model.predict(test_data)
print ("best train accuracy", svm_model.best_score_)
print ("test accuracy", accuracy_score(y_test, y_pred))

In [ ]:
print(classification_report(y_test, y_pred))

### SVM without SMOTE

In [ ]:
grid = tune_svm(data_prepared, y_train)

In [ ]:
y_pred = grid.predict(test_data)
print ("best train accuracy", grid.best_score_)
print ("test accuracy", accuracy_score(y_test, y_pred))

In [ ]:
print(classification_report(y_test, y_pred))

## K nearest neighbors

In [ ]:
# knn = KNeighborsClassifier()
# k_range = list(range(1, 31))
# param_grid = dict(n_neighbors=k_range)
# grid = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy')
# grid.fit(X, y)

In [ ]:
# knn_model = grid
# import joblib
# joblib.dump(knn_model, "telco_knn_model.pkl")

In [ ]:
knn_model.best_params_

In [ ]:
y_pred = knn_model.predict(test_data)
print ("best train accuracy", knn_model.best_score_)
print ("test accuracy", accuracy_score(y_test, y_pred))

In [ ]:
print(classification_report(y_test, y_pred))

### KNN Without SMOTE sampling

In [ ]:
# knn = KNeighborsClassifier()
# k_range = list(range(1, 31))
# param_grid = dict(n_neighbors=k_range)
# grid = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy')
# grid.fit(data_prepared, y_train)

In [ ]:
y_pred = knn_basemodel.predict(test_data)
print ("best train accuracy", knn_basemodel.best_score_)
print ("test accuracy", accuracy_score(y_test, y_pred))

In [ ]:
print(classification_report(y_test, y_pred))

# Logistic Regression

In [ ]:
def tune_log_reg(x,y): 
  logistic = linear_model.LogisticRegression(solver='liblinear')

  # Create regularization penalty space
  penalty = ['l1', 'l2']

  # Create regularization hyperparameter space
  C = [100, 10, 1.0, 0.1, 0.01]

  # Create hyperparameter options
  hyperparameters = dict(C=C, penalty=penalty)

  clf = GridSearchCV(logistic, hyperparameters, cv=5, verbose=0)
  best_model = clf.fit(x, y)
  return best_model


In [ ]:
#best_model = tune_log_reg(X,y)

In [ ]:
# logistic_model = best_model
# import joblib
# joblib.dump(logistic_model, "telco_logistic_model.pkl")

In [ ]:
log_model.best_params_

In [ ]:
y_logistic_pred = log_model.predict(test_data)
print ("best train accuracy", log_model.best_score_)
print ("test accuracy", accuracy_score(y_test, y_pred))

In [ ]:
print(classification_report(y_test, y_logistic_pred))

### Logistic regression without SMOTE sampling

In [ ]:
#best_model = tune_log_reg(data_prepared,y_train)

In [ ]:
y_pred = log_basemodel.predict(test_data)
print ("best train accuracy", log_basemodel.best_score_)
print ("test accuracy", accuracy_score(y_test, y_pred))

In [ ]:
print(classification_report(y_test, y_pred))

# Naive Bayes Classifier

In [ ]:
# #Create a Gaussian Classifier
# model = GaussianNB()

# # Train the model using the training sets
# model.fit(X,y)

In [ ]:
y_naive_pred = nb_model.predict(test_data)
accuracy_score(y_test, y_naive_pred)

In [ ]:
print(classification_report(y_test, y_naive_pred))

### Naive Bayes without SMOTE

In [ ]:
# #Create a Gaussian Classifier
# model = GaussianNB()

# # Train the model using the training sets
# model.fit(data_prepared,y_train)

In [ ]:
y_naive_pred = nb_basemodel.predict(test_data)
accuracy_score(y_test, y_naive_pred)

In [ ]:
print(classification_report(y_test, y_naive_pred))

# Random Forest Classifier

In [ ]:
def tune_random_forest(x,y):
  model_params = {
      'n_estimators': [50, 150, 250],
      'max_features': ['sqrt', 0.25, 0.5, 0.75, 1.0],
      'min_samples_split': [2, 4, 6]
  }

  # create random forest classifier model
  rf_model = RandomForestClassifier(random_state=1, oob_score = True)

  # set up grid search meta-estimator
  clf = GridSearchCV(rf_model, model_params, cv=5)

  # train the grid search meta-estimator to find the best mode
  clf.fit(x, y)
  return clf

#RF_model = tune_random_forest(X,y)

In [ ]:
RF_model.best_params_

In [ ]:
y_forest_pred = RF_model.predict(test_data)
print ("best train accuracy", RF_model.best_score_)
print ("test accuracy", accuracy_score(y_test, y_pred))

In [ ]:
print(classification_report(y_test, y_forest_pred))

### RF without SMOTE

In [ ]:
#RF_basemodel = tune_random_forest(data_prepared,y_train)

In [ ]:
y_forest_pred = RF_basemodel.predict(test_data)
print ("best train accuracy", RF_basemodel.best_score_)
print ("test accuracy", accuracy_score(y_test, y_pred))

In [ ]:
print(classification_report(y_test, y_forest_pred))

# Neural Network Classifier

In [ ]:
def FindLayerNodesLinear(n_layers, first_layer_nodes, last_layer_nodes):
    layers = []

    nodes_increment = (last_layer_nodes - first_layer_nodes)/ (n_layers-1)
    nodes = first_layer_nodes
    for i in range(1, n_layers+1):
        layers.append(math.ceil(nodes))
        nodes = nodes + nodes_increment

    return layers

def createmodel(n_layers, first_layer_nodes, last_layer_nodes, activation_func):
    model = Sequential()
    n_nodes = FindLayerNodesLinear(n_layers, first_layer_nodes, last_layer_nodes)
    for i in range(1, n_layers):
        if i==1:
            model.add(Dense(first_layer_nodes, input_dim=X.shape[1], activation=activation_func))
        else:
            model.add(Dense(n_nodes[i-1], activation=activation_func))

    #Finally, the output layer should have a single node in binary classification
    model.add(Dense(1, activation=activation_func))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ["accuracy"]) #note: metrics could also be 'mse'

    return model

#Wrap model into scikit-learn
model =  KerasClassifier(build_fn=createmodel, verbose = False)

activation_funcs = ['sigmoid', 'relu', 'tanh']
#loss_funcs = ['binary_crossentropy','hinge']
param_grid = dict(n_layers=[2,3], first_layer_nodes = [64,32,16], last_layer_nodes = [4], 
                  activation_func = activation_funcs, batch_size = [100], epochs = [20,60])
MLP = GridSearchCV(estimator = model, param_grid = param_grid)
MLP.fit(X,y)

In [ ]:
MLP.best_params_

In [ ]:
y_mlp_pred = MLP.predict(test_data)
print ("best train accuracy", MLP.best_score_)
print ("test accuracy", accuracy_score(y_test, y_pred))

In [ ]:
print(classification_report(y_test, y_mlp_pred))

### Neural Network without SMOTE

In [ ]:
#Wrap model into scikit-learn
model =  KerasClassifier(build_fn=createmodel, verbose = False)

activation_funcs = ['sigmoid', 'relu', 'tanh']
#loss_funcs = ['binary_crossentropy','hinge']
param_grid = dict(n_layers=[2,3], first_layer_nodes = [64,32,16], last_layer_nodes = [4], 
                  activation_func = activation_funcs, batch_size = [100], epochs = [20,60])
base_mlp = GridSearchCV(estimator = model, param_grid = param_grid)
base_mlp.fit(data_prepared, y_train)

In [ ]:
# Classification accuracy on test data
y_pred = base_mlp.predict(test_data)
print ("best train accuracy", base_mlp.best_score_)
print ("test accuracy", accuracy_score(y_test, y_pred))

# Precision Recall AUC plot

In [ ]:
# Create the SVM model with the best parameters again, as 'probability=True' was not there while hyperparameter tuning,
# which is needed for the function 'predict_proba'. 
print (svm_model.best_params_)
print (grid.best_params_)

In [ ]:
from sklearn.svm import SVC
svm = SVC(kernel='rbf', C=1, gamma=1, probability=True).fit(X,y)
svm_base = SVC(kernel='rbf', C=10, gamma=0.01, probability=True).fit(data_prepared,y_train)

In [ ]:
y_svm_pred = svm.predict(test_data)
accuracy_score(y_test, y_svm_pred)

In [ ]:
y_svm_pred = svm_base.predict(test_data)
accuracy_score(y_test, y_svm_pred)

In [ ]:
pred_prob1 = RF_model.predict_proba(test_data)
pred_prob2 = knn_model.predict_proba(test_data)
pred_prob3 = log_model.predict_proba(test_data)
pred_prob4 = nb_model.predict_proba(test_data)
pred_prob5 = svm.predict_proba(test_data)
pred_prob6 = MLP.predict_proba(test_data)

pred_baseprob1 = RF_basemodel.predict_proba(test_data)
pred_baseprob2 = knn_basemodel.predict_proba(test_data)
pred_baseprob3 = log_basemodel.predict_proba(test_data)
pred_baseprob4 = nb_basemodel.predict_proba(test_data)
pred_baseprob5 = svm_base.predict_proba(test_data)
pred_baseprob6 = base_mlp.predict_proba(test_data)

## Precision Recall Curves

In [ ]:
lr_precision1, lr_recall1, _ = precision_recall_curve(y_test, pred_prob1[:,1], pos_label='Yes')
lr_precision2, lr_recall2, _ = precision_recall_curve(y_test, pred_prob2[:,1], pos_label='Yes')
lr_precision3, lr_recall3, _ = precision_recall_curve(y_test, pred_prob3[:,1], pos_label='Yes')
lr_precision4, lr_recall4, _ = precision_recall_curve(y_test, pred_prob4[:,1], pos_label='Yes')
lr_precision5, lr_recall5, _ = precision_recall_curve(y_test, pred_prob5[:,1], pos_label='Yes')
lr_precision6, lr_recall6, _ = precision_recall_curve(y_test, pred_prob6[:,1], pos_label='Yes')

lr_precision7, lr_recall7, _ = precision_recall_curve(y_test, pred_baseprob1[:,1], pos_label='Yes')
lr_precision8, lr_recall8, _ = precision_recall_curve(y_test, pred_baseprob2[:,1], pos_label='Yes')
lr_precision9, lr_recall9, _ = precision_recall_curve(y_test, pred_baseprob3[:,1], pos_label='Yes')
lr_precision10, lr_recall10, _ = precision_recall_curve(y_test, pred_baseprob4[:,1], pos_label='Yes')
lr_precision11, lr_recall11, _ = precision_recall_curve(y_test, pred_baseprob5[:,1], pos_label='Yes')
lr_precision12, lr_recall12, _ = precision_recall_curve(y_test, pred_baseprob6[:,1], pos_label='Yes')

fig, axes = plt.subplots(2, 1, figsize=(15,20))
axes[0].plot(lr_recall1, lr_precision1, linestyle='--',color='orange', label='Random Forest')
axes[0].plot(lr_recall2, lr_precision2, linestyle='--',color='green', label='KNN')
axes[0].plot(lr_recall3, lr_precision3, linestyle='--',color='blue', label='Logistic Regression')
axes[0].plot(lr_recall4, lr_precision4, linestyle='--',color='red', label='Naive Bayes')
axes[0].plot(lr_recall5, lr_precision5, linestyle='--',color='yellow', label='SVM')
axes[0].plot(lr_recall6, lr_precision6, linestyle='--',color='brown', label='Neural Network')
axes[0].set_ylim([0.0, 1])
# title
axes[0].set_title('SMOTE PR curve')
# x label
axes[0].set_xlabel('Recall')
# y label
axes[0].set_ylabel('Precision')
axes[0].legend(loc='best')

axes[1].plot(lr_recall7, lr_precision7, linestyle='--',color='orange', label='Random Forest')
axes[1].plot(lr_recall8, lr_precision8, linestyle='--',color='green', label='KNN')
axes[1].plot(lr_recall9, lr_precision9, linestyle='--',color='blue', label='Logistic Regression')
axes[1].plot(lr_recall10, lr_precision10, linestyle='--',color='red', label='Naive Bayes')
axes[1].plot(lr_recall11, lr_precision11, linestyle='--',color='yellow', label='SVM')
axes[1].plot(lr_recall12, lr_precision12, linestyle='--',color='brown', label='Neural Network')

# title
axes[1].set_title('PR curve')
# x label
axes[1].set_xlabel('Recall')
# y label
axes[1].set_ylabel('Precision')
axes[1].legend(loc='best')

### We can see that the models are performing better when we are NOT doing BorderlineSMOTE sampling.
### Hence, we will continue with the models trained on the regular dataset(basemodels).

### Also, the probabilities generated by SVM and Random forest models can be calibrated by the 'CalibratedClassifierCV' method from sklearn.

In [ ]:
# reliability diagram
fop1, mpv1 = calibration_curve(y_test, pred_baseprob1[:,1], n_bins=10, normalize=True)
fop2, mpv2 = calibration_curve(y_test, pred_baseprob2[:,1], n_bins=10, normalize=True)
# plot perfectly calibrated
pyplot.plot([0, 1], [0, 1], linestyle='--')
# plot model reliability
pyplot.plot(mpv1, fop1, marker='.')
pyplot.plot(mpv2, fop2, marker='.')
pyplot.show()

### We can see that the both curves are mostly under the curve, meaning that the models have over-forecast i.e., the probabilites are too large.

In [ ]:
rf_cal = RF_basemodel.best_estimator_
rf_calibrated = CalibratedClassifierCV(rf_cal, method='sigmoid', cv=5)
rf_calibrated.fit(data_prepared, y_train)
# predict probabilities
probs_rf = rf_calibrated.predict_proba(test_data)[:, 1]
# reliability diagram
fop3, mpv3 = calibration_curve(y_test, probs_rf, n_bins=10, normalize=True)

svm_cal = svm_base
svm_calibrated = CalibratedClassifierCV(svm_cal, method='sigmoid', cv=5)
svm_calibrated.fit(data_prepared, y_train)
# predict probabilities
probs_svm = svm_calibrated.predict_proba(test_data)[:, 1]
# reliability diagram
fop4, mpv4 = calibration_curve(y_test, probs_svm, n_bins=10, normalize=True)

# plot perfectly calibrated
pyplot.plot([0, 1], [0, 1], linestyle='--')
# plot model reliability
pyplot.plot(mpv3, fop3, marker='.')
pyplot.plot(mpv4, fop4, marker='.')
pyplot.show()

### After calibration, we can see that parts of the curves are both above and below the base curve, making them much more balanced.
### Let's make a new PR curve with the calibrated probabilities:-

In [ ]:
lr_precision7, lr_recall7, _ = precision_recall_curve(y_test, probs_rf, pos_label='Yes')
lr_precision8, lr_recall8, _ = precision_recall_curve(y_test, pred_baseprob2[:,1], pos_label='Yes')
lr_precision9, lr_recall9, _ = precision_recall_curve(y_test, pred_baseprob3[:,1], pos_label='Yes')
lr_precision10, lr_recall10, _ = precision_recall_curve(y_test, pred_baseprob4[:,1], pos_label='Yes')
lr_precision11, lr_recall11, _ = precision_recall_curve(y_test, probs_svm, pos_label='Yes')
lr_precision12, lr_recall12, _ = precision_recall_curve(y_test, pred_baseprob6[:,1], pos_label='Yes')

#fig, axes = plt.subplots(2, 1, figsize=(12, 16))
plt.figure(figsize=(15,10))
plt.plot(lr_recall7, lr_precision7, linestyle='--',color='orange', label='Random Forest')
plt.plot(lr_recall8, lr_precision8, linestyle='--',color='green', label='KNN')
plt.plot(lr_recall9, lr_precision9, linestyle='--',color='blue', label='Logistic Regression')
plt.plot(lr_recall10, lr_precision10, linestyle='--',color='red', label='Naive Bayes')
plt.plot(lr_recall11, lr_precision11, linestyle='--',color='yellow', label='SVM')
plt.plot(lr_recall12, lr_precision12, linestyle='--',color='brown', label='Neural Network')
plt.ylim([0.0, 1])
# title
plt.title('PR curve')
# x label
plt.xlabel('Recall')
# y label
plt.ylabel('Precision')
plt.legend(loc='best')

## Voting Classifier

### We can also employ an ensemble learning method, and compare its performance with the previous models.

In [ ]:
a = y_test.to_list()
pred_base1 = RF_basemodel.predict(test_data)
pred_base2 = knn_basemodel.predict(test_data)
pred_base3 = log_basemodel.predict(test_data)
pred_base4 = nb_basemodel.predict(test_data)
pred_base5 = svm_base.predict(test_data)
pred_base6 = base_mlp.predict(test_data)

auc_score7 = round(auc(lr_recall7, lr_precision7),2)
rf_f1 = round(f1_score(a, pred_base2, pos_label='Yes'),2)

auc_score8 = round(auc(lr_recall8, lr_precision8),2)
knn_f1 = round(f1_score(a, pred_base2, pos_label='Yes'),2)

auc_score9 = round(auc(lr_recall9, lr_precision9),2)
log_reg_f1 = round(f1_score(a, pred_base3, pos_label='Yes'),2)

auc_score10 = round(auc(lr_recall10, lr_precision10),2)
nb_f1 = round(f1_score(a, pred_base4, pos_label='Yes'),2)

auc_score11 = round(auc(lr_recall11, lr_precision11),2)
svm_f1 = round(f1_score(a, pred_base5, pos_label='Yes'),2)

auc_score12 = round(auc(lr_recall12, lr_precision12),2)
mlp_f1 = round(f1_score(a, pred_base6, pos_label='Yes'),2)

In [ ]:
voting_clf = VotingClassifier(estimators=[('RF', RF_model.best_estimator_), ('knn', knn_model.best_estimator_), ('LogReg', log_model.best_estimator_), ('NB', nb_model),
                                          ('SVM', svm_base)], voting='soft')
voting_clf.fit(data_prepared, y_train)
preds = voting_clf.predict(test_data)

a = y_test.to_list()
b = preds.tolist()
acc = accuracy_score(a, b)
f1 = round(f1_score(a, b, pos_label='Yes'),2)

# PR AUC score
probs = voting_clf.predict_proba(test_data)
model_probs = probs[:, 1]
# calculate the precision-recall auc
precision, recall, _ = precision_recall_curve(y_test, model_probs, pos_label='Yes')
auc_score = round(auc(recall, precision),2)
print ("Statistics for the voting classsifier:-")
print("Accuracy is: " + str(acc))
print("F1 Score is: " + str(f1))
print('Voting classifier PR AUC: %.3f' % auc_score)

### Plot the F1 scores and Precision-Recall AUC value:-

In [ ]:
models = ['Random Forest', 'KNN', 'Log Reg', 'Naive Bayes', 'SVM', 'Neural Network', 'Ensemble model']
F1_scores = [rf_f1, knn_f1, log_reg_f1, nb_f1, svm_f1, mlp_f1, f1]
pr_auc = [auc_score7, auc_score8, auc_score9, auc_score10, auc_score11, auc_score12, auc_score]

x = np.arange(len(models))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots(figsize=(12,8))
rects1 = ax.bar(x - width/2, F1_scores, width, label='F1 scores')
rects2 = ax.bar(x + width/2, pr_auc, width, label='PR AUC')

# Add some text for labels, title and custom x-axis tick labels, etc.
#fig.figure()
ax.set_ylim([0.0, 1])
ax.set_ylabel('Scores')
ax.set_title('Scores by F1 score and PR AUC')
ax.set_xticks(x)
ax.set_xticklabels(models)
ax.legend(loc='best')


def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')


autolabel(rects1)
autolabel(rects2)

fig.tight_layout()

#plt.show()

## Out of all the 7 models trained, the neural network model is providing the best combination of PR AUC, and F1 scores. Hence, we can finalize that as the best algorithm for this dataset.

In [ ]:
y_mlp_pred = base_mlp.predict(test_data)

print("Statistics for the neural network model:-")
print("Accuracy is: " + str(accuracy_score(y_test, y_mlp_pred)))
print("F1 Score is: " + str(mlp_f1))
print('Voting classifier PR AUC: %.3f' % auc_score12)